In [2]:
import numpy as np
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests
import re
import os

from bsoup_scrape import bsoup_scrape_data
from selenium_scrape import scrape_page2

In [2]:
titles = []
years = []
episodes = []
countries = []
scores = []
ranks = []

titles_p2 = []
genres = []
network = []
watchers = []

# Part 1 - Beautiful Soup

This section of code will use beautiful soup to scrape everything shown on the first page (the one listing all the titles)

In [42]:
# URL to scrape
url_base = 'https://mydramalist.com/shows/top?page='
page_number = "27"
url = url_base+page_number
# Send a GET request to the URL
response = requests.get(url)
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

bsoup_scrape_data(titles, years, episodes, countries, scores, ranks, soup)

my_drama_list = pd.DataFrame({
    "title": titles,
    "year": years,
    "episodes": episodes,
    "country": countries,
    "viewer_score": scores,
    "rank": ranks
})
soup_scrape = pd.DataFrame(my_drama_list)

#if the csv file already exists, delete it
#if os.path.exists("msoup_scrapedl.csv"):
   #os.remove("soup_scrape.csv")

# soup_scrape.to_csv("combined_soup.csv", index=False) #commenting this out so I don't accidentally delete it again

# Part 2 - Selenium

This section of the code will use selenium to click through each title page and scrape additional elements

In [3]:
url = 'https://mydramalist.com/shows/top?page=15'
scrape_page2(titles_p2, genres, watchers, network, url)

selenium_scrape = pd.DataFrame({
    "title": titles_p2,
    "genres": genres,
    "network": network,
    "watchers": watchers,
})
ss_df = pd.DataFrame(selenium_scrape)
ss_df

ss_df.to_csv("deleteme.csv", index=False)

timeout error- getting url
It's Okay, That's Friendship
Timeout error- clicking title 
No elements with 'original network' text found
timeout error - going back
The Princess's Man
Timeout error- clicking title 
timeout error - going back
Voice Season 2
Timeout error- clicking title 
timeout error - going back
Forever and Ever
Timeout error- clicking title 
Liar Game 2
Recipe for Farewell
Timeout error- clicking title 
No elements with 'original network' text found
Doom at Your Service
Timeout error- clicking title 
timeout error - going back
F4 Thailand: Boys Over Flowers
Timeout error- clicking title 
Happy Birthday
Timeout error- clicking title 
timeout error - going back
Because This Is My First Life
Timeout error- clicking title 
timeout error - going back
Dali and the Cocky Prince
Timeout error- clicking title 
timeout error - going back
Lost
Timeout error- clicking title 
timeout error - going back
Seasons of Blossom
Timeout error- clicking title 
No elements with 'original netwo

IndexError: list index out of range

## Merging The DataFrames
This section of the code will merge the csv files created from part 1 and part 2

In [1]:
# Read the CSV files
import pandas as pd
import os
ss = pd.read_csv('combined_selenium.csv')
ss19 = pd.read_csv('ss_df19.csv')
ss20 = pd.read_csv('ss_df20.csv')
ss21 = pd.read_csv('ss_df21.csv')
ss22 = pd.read_csv('ss_df22.csv')
ss23 = pd.read_csv('ss_df23.csv')
ss24 = pd.read_csv('ss_df24.csv')
ss25 = pd.read_csv('ss_df25.csv')
ss26 = pd.read_csv('ss_df26.csv')


# Combine the files
selenium_mdl = pd.concat([ss, ss19, ss20, ss21, ss22, ss23, ss24,ss25,ss26])
# if the csv file already exists, delete it
if os.path.exists("combined_selenium.csv"):
    os.remove("combined_selenium.csv")
# Write the combined data to a new CSV file
selenium_mdl.to_csv('combined_selenium.csv', index=False)


merge the selenium and beautiful soup csv files

In [163]:
soup = pd.read_csv('combined_soup.csv')
sel = pd.read_csv('combined_selenium.csv')

# Merge based on the 'title' column
merged = pd.merge(soup, sel, on='title', how='inner') #'inner' keeps only matching rows

# if the csv file already exists, delete it
if os.path.exists("combined_mdl.csv"):
    os.remove("combined_mdl.csv")

# Write the merged data to a new CSV file
merged.to_csv('combined_mdl.csv', index=False)

## Polishing
This section puts the finishing touches on the final csv file :D

In [3]:
mdl = pd.read_csv('combined_mdl.csv')
test = mdl
# clean up the network and genres column:
with pd.option_context('mode.chained_assignment', None):
    test['network'] = test['network'].str.split(': ').str[1]
    test['genres'] = test['genres'].astype(str).str.replace(", ", ",")
    test['genres'] = test['genres'].str.split(': ').str[1]
    test['genres'] = test['genres'].astype(str).str.replace(", ", ",")


Currently, the genre and network are input as long strings. I am going to split them up into their own columns with a 1 or 0 if present.

In [4]:
# collect all unique genres present in the df before splitting the column into separate columns
unique_genres = set()
for genres in test['genres']:
    split_genres = genres.split(',')
    for genre in split_genres:
        unique_genres.add(genre)

unique_network = set()
for network in test['network']:
    if pd.notnull(network) and network != 'na':  # Check for non-null and non-'na' values
        split_network = network.split(',')
        for option in split_network:
            unique_network.add(option)

In [9]:
type(unique_genres)

set

In [5]:
# Creating columns for each unique genre/network, put a zero if it's null/na
with pd.option_context('mode.chained_assignment', None):
    for genre in unique_genres:
        test.loc[:, genre.strip()] = test['genres'].apply(lambda x: 1 if pd.notnull(x) and x != 'na' and genre in x else 0)
    
    for network in unique_network:
        test.loc[:, network.strip()] = test['network'].apply(lambda x: 1 if pd.notnull(x) and x != 'na' and network in x else 0)

    # Dropping the original 'genres' column
    #test.drop('genres', axis=1, inplace=True)
    #test.drop('network', axis=1, inplace=True)

In [6]:
test.to_csv("mdl_final.csv", index=False)

In [7]:
print(test.shape)

sums = test.iloc[:, 8:].sum(axis=0)
# Get columns to drop where the sum is less than 2
columns_to_drop = sums[sums < 16].index

# Drop columns where the sum is less than 2
test2 = test.drop(columns=columns_to_drop)
print(test2.shape)


(523, 94)
(523, 39)


In [204]:
test2.to_csv("mdl_final.csv", index=False)

hooray we made it!

In [1]:
import pandas as pd
aaa = pd.read_csv('mdl_final.csv')
aaa.columns[9:40]

Index(['Crime', 'Sci-Fi', 'Wuxia', 'Youth', 'Medical', 'Psychological',
       'Historical', 'Romance', 'Military', 'Music', 'Business', 'Thriller',
       'Fantasy', 'Sports', 'Melodrama', 'Supernatural', 'Family', 'Tokusatsu',
       'Law', 'Mystery', 'Adventure', 'Food', 'Horror', 'Life', 'Sitcom',
       'Comedy', 'Political', 'Drama', 'Mature', 'Action', 'War'],
      dtype='object')

In [14]:
aaa.iloc[:, 40:]

,Netflix,tvN,jTBC,TBS,JSTV,Viki,WeTV,TVK,GDTV,NTV,...,Channel A,Hulu,Disney+,TVING,TTV,MBN,iQiyi,GMM One,Fuji TV,BTV
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
519,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
520,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
521,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Calculate the sum of each column
df = aaa.iloc[:, 60:]
column_sums = df.sum()

# Select column names that have a sum of at least 50
selected_columns = column_sums[column_sums >= 15].index.tolist()


print(column_sums)

Daum Kakao TV     2
WOWOW             3
Tencent Video    19
Dragon TV        11
Amazon Prime      2
ZJTV              7
ENA               6
PTS               1
PPTV              1
COUPANG TV        2
ViuTV             8
KBS2             27
Mango TV         14
CTV               9
Youku            40
SBS              38
DRAMAcube         1
GMM 25           14
True4U            1
OCN               2
MBC              29
Channel 3         4
SET TV            1
Sohu TV           1
Channel A         1
Hulu              7
Disney+          12
TVING            12
TTV               1
MBN               2
iQiyi            53
GMM One           5
Fuji TV          19
BTV               5
dtype: int64
